## Method
- Make prediction for localidad level (Ciudad Bolívar) using experimental design 2, Naive Counting and SEPPexp based on week-days model.
- Compare model performance using hit rate traditional implementation, true positives proportion and PAI.

## Hypothesis:
SEPP exp prediction based on week-days performance is better than the obtained with counting model.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
from services.prediction_experiment import PredictionExperiment

Failed to import `rtree`.
Failed to import `rtree`.


In [2]:
from services import prediction_metrics

In [3]:
def get_hit_rate_from_dict(row,column,coverage):
    return row[column][coverage]

## SIEDCO

In [4]:
csv_path = '/Users/anamaria/Desktop/dev/security_project/datasets/deduplicate_siedco_09062020.csv'
siedco_info = {'name':'SIEDCO','path':csv_path}

## Counting

In [5]:
train_dates = {'initial':'2018-03-01','final':'2018-09-30'}
validation_dates = {'initial':'2018-10-01','final':'2018-10-07'}
metrics = ''
aggregation = ''

In [6]:
filter_localidad = {'field':'LOCALIDAD','value':'CIUDAD BOLIVAR'}
model = "NaiveCounting"
localidad_experiment = PredictionExperiment(dataset_info=siedco_info, custom_filter=filter_localidad,train_dates=train_dates, validation_dates=validation_dates, model=model,metrics='',aggregation_data='')
prediction_array = localidad_experiment.run_ncv_experiment(time_unit='',grid_size=150, outer_iterations='')

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (32,44) have mixed types. Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 9, 30, 0, 0)}
current_validation_date 2018-10-01 00:00:00
1203
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 1, 0, 0)}
current_validation_date 2018-10-02 00:00:00
1206
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 2, 0, 0)}
current_validation_date 2018-10-03 00:00:00
1210
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 3, 0, 0)}
current_validation_date 2018-10-04 00:00:00
1213
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 4, 0, 0)}
current_validation_date 2018-10-05 00:00:00
1218
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 5, 0, 0)}
current_validation_date 2018-10-06 00:00:00
1226
train_subset_dat

In [7]:
df_counting = pd.DataFrame(prediction_array, columns =['initial-date','final-date','prediction','eval_pts'])
df_result = df_counting.copy()

In [8]:
coverages = [2,4,6,8,10,12,14,16,18,20]
df_result['hitrate_default'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'default'), axis=1)
df_result['hitrate_TP'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'ground_truth_coverage'), axis=1)


In [9]:
##For hitrate_TP "true positives"
df_result['coverage_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].keys())[0], axis=1)
df_result['hit_rate_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].values())[0], axis=1)
df_result['PAI_TP'] = df_result['hit_rate_TP'] / (df_result['coverage_TP']/100)

In [10]:
##For hitrate_default
coverages = [2.0,4.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0]
column_dict = 'hitrate_default'
for c in coverages:
    new_hit_rate_column = 'hit_rate_default_coverage_'+str(c)
    df_result[new_hit_rate_column] = df_result.apply(lambda row: get_hit_rate_from_dict(row,column_dict,c), axis=1)

    ##PAI
    new_column = 'PAI_default_coverage_'+str(c)
    df_result[new_column] = df_result[new_hit_rate_column]/(c/100)


In [11]:
#delete datetimes where no crimes were reported (0 crimes on ground truth -> hit-rate = -1)
df_result = df_result[df_result['hit_rate_default_coverage_2.0']!= -1]
print(df_result.mean())

coverage_TP                       0.072962
hit_rate_TP                       0.000000
PAI_TP                            0.000000
hit_rate_default_coverage_2.0     0.083333
PAI_default_coverage_2.0          4.166667
hit_rate_default_coverage_4.0     0.199074
PAI_default_coverage_4.0          4.976852
hit_rate_default_coverage_6.0     0.273148
PAI_default_coverage_6.0          4.552469
hit_rate_default_coverage_8.0     0.300926
PAI_default_coverage_8.0          3.761574
hit_rate_default_coverage_10.0    0.398148
PAI_default_coverage_10.0         3.981481
hit_rate_default_coverage_12.0    0.500000
PAI_default_coverage_12.0         4.166667
hit_rate_default_coverage_14.0    0.518519
PAI_default_coverage_14.0         3.703704
hit_rate_default_coverage_16.0    0.574074
PAI_default_coverage_16.0         3.587963
hit_rate_default_coverage_18.0    0.587963
PAI_default_coverage_18.0         3.266461
hit_rate_default_coverage_20.0    0.703704
PAI_default_coverage_20.0         3.518519
dtype: floa

## Counting (reduce base dates)
This approach is proposed to keep a similar number of training points (considering this model 'NaiveCounting' and background kernel on sepp the prediction is related to the historical spatial distribution/concentration of events) 

In [5]:
train_dates = {'initial':'2018-07-01','final':'2018-09-30'}
validation_dates = {'initial':'2018-10-01','final':'2018-10-07'}
metrics = ''
aggregation = ''

In [6]:
filter_localidad = {'field':'LOCALIDAD','value':'CIUDAD BOLIVAR'}
model = "NaiveCounting"
localidad_experiment = PredictionExperiment(dataset_info=siedco_info, custom_filter=filter_localidad,train_dates=train_dates, validation_dates=validation_dates, model=model,metrics='',aggregation_data='')
prediction_array = localidad_experiment.run_ncv_experiment(time_unit='',grid_size=150, outer_iterations='')

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (32,44) have mixed types. Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


train_subset_dates {'initial': datetime.datetime(2018, 7, 1, 0, 0), 'final': datetime.datetime(2018, 9, 30, 0, 0)}
current_validation_date 2018-10-01 00:00:00
536
train_subset_dates {'initial': datetime.datetime(2018, 7, 1, 0, 0), 'final': datetime.datetime(2018, 10, 1, 0, 0)}
current_validation_date 2018-10-02 00:00:00
539
train_subset_dates {'initial': datetime.datetime(2018, 7, 1, 0, 0), 'final': datetime.datetime(2018, 10, 2, 0, 0)}
current_validation_date 2018-10-03 00:00:00
543
train_subset_dates {'initial': datetime.datetime(2018, 7, 1, 0, 0), 'final': datetime.datetime(2018, 10, 3, 0, 0)}
current_validation_date 2018-10-04 00:00:00
546
train_subset_dates {'initial': datetime.datetime(2018, 7, 1, 0, 0), 'final': datetime.datetime(2018, 10, 4, 0, 0)}
current_validation_date 2018-10-05 00:00:00
551
train_subset_dates {'initial': datetime.datetime(2018, 7, 1, 0, 0), 'final': datetime.datetime(2018, 10, 5, 0, 0)}
current_validation_date 2018-10-06 00:00:00
559
train_subset_dates {'i

In [7]:
df_counting_2 = pd.DataFrame(prediction_array, columns =['initial-date','final-date','prediction','eval_pts'])
df_result = df_counting_2.copy()

In [8]:
coverages = [2,4,6,8,10,12,14,16,18,20]
df_result['hitrate_default'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'default'), axis=1)
df_result['hitrate_TP'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'ground_truth_coverage'), axis=1)


In [9]:
##For hitrate_TP "true positives"
df_result['coverage_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].keys())[0], axis=1)
df_result['hit_rate_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].values())[0], axis=1)
df_result['PAI_TP'] = df_result['hit_rate_TP'] / (df_result['coverage_TP']/100)

In [10]:
##For hitrate_default
coverages = [2.0,4.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0]
column_dict = 'hitrate_default'
for c in coverages:
    new_hit_rate_column = 'hit_rate_default_coverage_'+str(c)
    df_result[new_hit_rate_column] = df_result.apply(lambda row: get_hit_rate_from_dict(row,column_dict,c), axis=1)

    ##PAI
    new_column = 'PAI_default_coverage_'+str(c)
    df_result[new_column] = df_result[new_hit_rate_column]/(c/100)


In [11]:
#delete datetimes where no crimes were reported (0 crimes on ground truth -> hit-rate = -1)
df_result = df_result[df_result['hit_rate_default_coverage_2.0']!= -1]
print(df_result.mean())

coverage_TP                        0.079952
hit_rate_TP                        0.027778
PAI_TP                            33.777778
hit_rate_default_coverage_2.0      0.046296
PAI_default_coverage_2.0           2.314815
hit_rate_default_coverage_4.0      0.185185
PAI_default_coverage_4.0           4.629630
hit_rate_default_coverage_6.0      0.185185
PAI_default_coverage_6.0           3.086420
hit_rate_default_coverage_8.0      0.226852
PAI_default_coverage_8.0           2.835648
hit_rate_default_coverage_10.0     0.282407
PAI_default_coverage_10.0          2.824074
hit_rate_default_coverage_12.0     0.356481
PAI_default_coverage_12.0          2.970679
hit_rate_default_coverage_14.0     0.555556
PAI_default_coverage_14.0          3.968254
hit_rate_default_coverage_16.0     0.574074
PAI_default_coverage_16.0          3.587963
hit_rate_default_coverage_18.0     0.574074
PAI_default_coverage_18.0          3.189300
hit_rate_default_coverage_20.0     0.574074
PAI_default_coverage_20.0       

## SEPPexp week-day

In [12]:
train_dates = {'initial':'2017-03-01','final':'2018-09-30'}
validation_dates = {'initial':'2018-10-01','final':'2018-10-07'}
metrics = ''
aggregation = ''

In [13]:
filter_localidad = {'field':'LOCALIDAD','value':'CIUDAD BOLIVAR'}
model = "SEPPexpWeekDay"
localidad_experiment = PredictionExperiment(dataset_info=siedco_info, custom_filter=filter_localidad,train_dates=train_dates, validation_dates=validation_dates, model=model,metrics='',aggregation_data='')
prediction_array = localidad_experiment.run_ncv_experiment(time_unit='',grid_size=150, outer_iterations='')

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (32,44) have mixed types. Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


train_subset_dates {'initial': datetime.datetime(2017, 3, 1, 0, 0), 'final': datetime.datetime(2018, 9, 30, 0, 0)}
current_validation_date 2018-10-01 00:00:00


../services/aggressive_model.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_subset['weekday'] = df_train_subset['TIME_STAMP'].dt.day_name()


omega: 0.00487815773676205, theta: 0.013119736268712311
450
train_subset_dates {'initial': datetime.datetime(2017, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 1, 0, 0)}
current_validation_date 2018-10-02 00:00:00
omega: 0.06772236696416421, theta: 0.007921283332816692
372
train_subset_dates {'initial': datetime.datetime(2017, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 2, 0, 0)}
current_validation_date 2018-10-03 00:00:00
omega: 0.002955964248139879, theta: 0.016077897448847037
365
train_subset_dates {'initial': datetime.datetime(2017, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 3, 0, 0)}
current_validation_date 2018-10-04 00:00:00
omega: 0.020956520403629234, theta: 0.016088235152791272
367
train_subset_dates {'initial': datetime.datetime(2017, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 4, 0, 0)}
current_validation_date 2018-10-05 00:00:00
omega: 0.021468392925948803, theta: 0.010012979952576771
353
train_subset_dates {'initial': datetime.datetime(2017, 3, 1, 

In [14]:
df_seppexp = pd.DataFrame(prediction_array, columns =['initial-date','final-date','prediction','eval_pts'])
df_result = df_seppexp.copy()

In [15]:
coverages = [2,4,6,8,10,12,14,16,18,20]
df_result['hitrate_default'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'default'), axis=1)
df_result['hitrate_TP'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'ground_truth_coverage'), axis=1)


In [16]:
##For hitrate_TP "true positives"
df_result['coverage_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].keys())[0], axis=1)
df_result['hit_rate_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].values())[0], axis=1)
df_result['PAI_TP'] = df_result['hit_rate_TP'] / (df_result['coverage_TP']/100)

In [17]:
##For hitrate_default
coverages = [2.0,4.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0]
column_dict = 'hitrate_default'
for c in coverages:
    new_hit_rate_column = 'hit_rate_default_coverage_'+str(c)
    df_result[new_hit_rate_column] = df_result.apply(lambda row: get_hit_rate_from_dict(row,column_dict,c), axis=1)

    ##PAI
    new_column = 'PAI_default_coverage_'+str(c)
    df_result[new_column] = df_result[new_hit_rate_column]/(c/100)


In [18]:
#delete datetimes where no crimes were reported (0 crimes on ground truth -> hit-rate = -1)
df_result = df_result[df_result['hit_rate_default_coverage_2.0']!= -1]
print(df_result.mean())

coverage_TP                        0.078688
hit_rate_TP                        0.027778
PAI_TP                            37.333333
hit_rate_default_coverage_2.0      0.143519
PAI_default_coverage_2.0           7.175926
hit_rate_default_coverage_4.0      0.162037
PAI_default_coverage_4.0           4.050926
hit_rate_default_coverage_6.0      0.245370
PAI_default_coverage_6.0           4.089506
hit_rate_default_coverage_8.0      0.245370
PAI_default_coverage_8.0           3.067130
hit_rate_default_coverage_10.0     0.273148
PAI_default_coverage_10.0          2.731481
hit_rate_default_coverage_12.0     0.273148
PAI_default_coverage_12.0          2.276235
hit_rate_default_coverage_14.0     0.310185
PAI_default_coverage_14.0          2.215608
hit_rate_default_coverage_16.0     0.310185
PAI_default_coverage_16.0          1.938657
hit_rate_default_coverage_18.0     0.328704
PAI_default_coverage_18.0          1.826132
hit_rate_default_coverage_20.0     0.384259
PAI_default_coverage_20.0       